# Download images with Sunpy

Sunpy is a python package to access and make use of solar data.

We will use it to download SDO images

In [5]:
from sunpy.net import Fido, attrs as a
import astropy.units    as u

For each timestamp, we need to download a continuum intensity map, a Dopplergram, and a line-of-sight magnetogram, using the `search` function in `Fido`.

Example use:
```
matching_images = Fido.search(
    a.Time('2014-01-01T00:00:00', '2014-01-01T00:12:00'),
    a.jsoc.Series('hmi.Ic_720s'),
    a.jsoc.Notify('example@email.com')
)

```

`a.Time` specifies the time range in which we want to search for data. 

`a.jsoc.Series` tells jsoc which data products we want to access. In this case, we are getting the 720 second exposure (`720s`),  continuum intensitygrams (`Ic`) obtained from the Helioseismic and Magnetic Imager (`hmi`). To access the Dopplergrams we would use `'hmi.v_720s'` and for the 

`a.jsoc.Notify` tells jsoc which user is accessing the data, you will get a confirmation email each time you download the data.

Another useful parameter to pass to `seach` includes, e.g., `a.Sample(12*u.hour)` if we wanted to only get data every 12 hours for the duration of the specified time range.

Once we have identified all the matching available data products using `Fido.search`, we call

```
downloaded_files = Fido.fetch(matching_images, path='./path/{file}')
```
to download the relevant images, and save them in `./path/`. 

The `downloaded_files` object is a list of filepaths of the images.

In [18]:
continuum_search = Fido.search(a.Time('2014-01-01T00:00:00', '2014-01-01T00:12:00'), a.jsoc.Series('hmi.Ic_720s'), a.jsoc.Notify('bsl204@exeter.ac.uk'))
continuum_download = Fido.fetch(continuum_search, path='./downloaded_images/{file}')

magnetogram_search = Fido.search(a.Time('2014-01-01T00:00:00', '2014-01-01T00:12:00'), a.jsoc.Series('hmi.m_720s'), a.jsoc.Notify('bsl204@exeter.ac.uk'))
magnetogram_download = Fido.fetch(magnetogram_search, path='path='./downloaded_images/{file}')

dopplergram_search = Fido.search(a.Time('2014-01-01T00:00:00', '2014-01-01T00:12:00'), a.jsoc.Series('hmi.V_720s'), a.jsoc.Notify('bsl204@exeter.ac.uk'))
dopplergram_download = Fido.fetch(dopplergram_search, path='./{file}')

KeyboardInterrupt: 

In [15]:
print(dopplergram_download)

['hmi.v_720s.20140101_000000_TAI.1.Dopplergram.fits', 'hmi.v_720s.20140101_001200_TAI.1.Dopplergram.fits']


In [16]:
continuum_search

T_REC,TELESCOP,INSTRUME,WAVELNTH,CAR_ROT
str23,str7,str9,float64,int64
2014.01.01_00:00:00_TAI,SDO/HMI,HMI_SIDE1,6173.0,2145
2014.01.01_00:12:00_TAI,SDO/HMI,HMI_SIDE1,6173.0,2145
